In [1]:
#Importing all the necessary libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif

#Loading the dataset
df = pd.read_csv("C:\RHEA/S7/Project/Code/Datasets/Gastric_gene.csv")

#Encoding the target variable
le = LabelEncoder()
df["Sample_Characteristics"] = le.fit_transform(df["Sample_Characteristics"])

#Defining features and target
X = df.drop(columns=["Sample_Characteristics"])
y = df["Sample_Characteristics"]

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

X.head()


<>:17: SyntaxWarning: invalid escape sequence '\R'
<>:17: SyntaxWarning: invalid escape sequence '\R'
C:\Users\rheam\AppData\Local\Temp\ipykernel_19528\1212683252.py:17: SyntaxWarning: invalid escape sequence '\R'
  df = pd.read_csv("C:\RHEA/S7/Project/Code/Datasets/Gastric_gene.csv")


,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC02689,V58727,V58728,V58729,H2AQ1P,NBEAP6,V58732,V58733,V58734,V58735
0,1157,2,821,323,92,1152,3575,1574,1215,590,...,0,0,0,0,0,0,3,0,0,1
1,381,0,703,302,119,334,11342,1616,1268,477,...,0,0,0,0,0,0,3,0,0,2
2,1603,2,741,268,138,247,1558,1458,1044,542,...,0,0,1,0,0,0,5,0,0,0
3,718,0,422,300,79,164,1434,1386,1384,476,...,0,0,1,0,0,0,2,0,0,0
4,482,5,515,283,118,486,1485,1620,901,506,...,0,0,0,0,0,0,4,0,0,2


In [2]:
# No feature selection + XGB
# Initializing and train the XGBoost classifier
xgb_clf = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric="logloss")
xgb_clf.fit(X_train, y_train)

# Making predictions
y_pred = xgb_clf.predict(X_test)
y_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]  # Probability estimates for AUC calculation

# Evaluating model performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
kappa = cohen_kappa_score(y_test, y_pred)

# Printing evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Kappa: {kappa:.4f}")



Accuracy: 0.9353
Precision: 0.9286
AUC: 0.9861
Kappa: 0.8705


In [3]:
#XGB + RF
# Train XGBoost to get feature importances
xgb_selector = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6,
                             subsample=0.8, colsample_bytree=0.8, random_state=42,
                             use_label_encoder=False, eval_metric="logloss")

xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# List of feature selection counts
feature_counts = [10, 50, 100, 500, 1000]

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])

# Loop through different feature counts
for num_features in feature_counts:
    # Select top N features
    top_features = feature_importances.nlargest(num_features).index.tolist()

    # Filter dataset with selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]

    # Train Random Forest classifier
    rf_clf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2,
                                    random_state=42, n_jobs=-1)
    rf_clf.fit(X_train_selected, y_train)

    # Make predictions
    y_pred = rf_clf.predict(X_test_selected)
    y_pred_proba = rf_clf.predict_proba(X_test_selected)[:, 1]  # Probability estimates for AUC calculation

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Store results in DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],
                                                      columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])],
                                                      ignore_index=True)

# Display final results
print(results_df)
     

c:\RHEA\S7\Project\Code\Cancer-Classification\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:34:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rheam\AppData\Local\Temp\ipykernel_19528\1287839968.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],


  Features       AUC  Accuracy  Precision     Kappa
0       10  0.978571  0.935252   0.968750  0.870430
1       50  0.980228  0.956835   0.984615  0.913629
2      100  0.981470  0.956835   0.970149  0.913647
3      500  0.988716  0.949640   0.955882  0.899265
4     1000  0.985197  0.949640   0.955882  0.899265
